将训练集数据整理为以类划分的数据集，读取 train.csv 文件并创建一个新的数据集文件夹，其中每个类别都有自己的子文件夹

In [4]:
import os
import shutil
import pandas as pd

# 定义路径
csv_file = '/root/autodl-tmp/xin/Model-Bias-in-Training-with-Generated-Data/Dataset/train_labels.csv'
images_path = '/root/autodl-tmp/xin/Model-Bias-in-Training-with-Generated-Data/Dataset/Colored_MNIST_Bais'
target_base_path = '/root/autodl-tmp/xin/Model-Bias-in-Training-with-Generated-Data/Dataset/Colored_MNIST_Bais_Organized'

# 创建目标文件夹
if not os.path.exists(target_base_path):
    os.makedirs(target_base_path)

# 读取CSV文件
data = pd.read_csv(csv_file)

# 创建类别文件夹并移动文件
for index, row in data.iterrows():
    label = row['label']
    image_name = row['image_name']
    
    # 创建类别文件夹
    target_folder = os.path.join(target_base_path, str(label))
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # 原始图片路径
    src_image_path = os.path.join(images_path, image_name)
    
    # 目标图片路径
    dst_image_path = os.path.join(target_folder, image_name)
    
    # 移动图片
    shutil.copy(src_image_path, dst_image_path)

print("图片已成功组织到新的类别文件夹中。")


图片已成功组织到新的类别文件夹中。


In [3]:
import os
import shutil

# 定义路径
target_base_path = '/root/autodl-tmp/xin/Model-Bias-in-Training-with-Generated-Data/Dataset/Colored_MNIST_Bais_Organized'
images_path = '/root/autodl-tmp/xin/Model-Bias-in-Training-with-Generated-Data/Dataset/Colored_MNIST_Bais'

# 遍历目标文件夹中的所有图片
for root, dirs, files in os.walk(target_base_path):
    for file in files:
        if file.endswith('.png'):  # 确保只处理图片文件
            src_image_path = os.path.join(root, file)
            dst_image_path = os.path.join(images_path, file)
            
            # 复制图片回源文件夹
            shutil.copy(src_image_path, dst_image_path)

print("图片已成功复制回源文件夹。")


图片已成功复制回源文件夹。


查看颜色通道的std mean

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# 数据加载器
dataset = datasets.ImageFolder(root='/root/autodl-tmp/xin/datasets/CMNIST/data/train', transform=transforms.ToTensor())
loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)

# 计算均值和标准差
mean = 0.
std = 0.
n_samples = 0.

for images, _ in loader:
    batch_samples = images.size(0)  # batch size (current)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    n_samples += batch_samples

mean /= n_samples
std /= n_samples

print(f'Mean: {mean}')
print(f'Std: {std}')


Mean: tensor([0.0926, 0.0793, 0.0775])
Std: tensor([0.1854, 0.1746, 0.1583])
